In [1]:
'''If gives out error, ignore'''
from pyspark import SparkContext
sc=SparkContext()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [3]:
TWEET_PATH = 'D:/Datahack/bigdatathon-2017/data/'
TWEET_FILE = 'tweetssentiment.txt'
TWEET_PATHFILE = TWEET_PATH+TWEET_FILE
ITEM_PATH = 'D:/Datahack/bigdatathon-2017/scrapers/farfetch/'
ITEM_FILE = 'kv-data-hashed-forvictor'
ITEM_PATHFILE = ITEM_PATH+ITEM_FILE



Tweets = (sc.textFile(TWEET_PATHFILE, 4).distinct().map(lambda x : x.split(',')).cache())


In [4]:
Items = (sc.textFile(ITEM_PATHFILE, 4).map(lambda x : x.split('","')).cache())

In [5]:
stopwords = set(["a","b", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
                  "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone",
                  "anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming",
                  "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom",
                  "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
                  "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every",
                  "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly",
                  "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her",
                  "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if",
                  "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many",
                  "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself",
                  "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                  "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
                  "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem",
                  "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                  "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the",
                  "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv",
                  "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward",
                  "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were",
                  "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while",
                  "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you",
                  "your", "yours", "yourself", "yourselves", "the","add","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]);


In [94]:
import re
split_regex = r'[a-zA-Z0-9_]+'
def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    string = re.sub(r'([^\s\w(?<_)])+', ' ',string.lower())
    return re.findall(split_regex, string)


def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    string = removeQuotes(string)
    def remove_if_not_substring(l1, l2):
        return [i for i in l1 if not any(j == i for j in list(l2))]
    string = remove_if_not_substring(simpleTokenize(string), stopwords)
    return string



TweetsPARSED = Tweets.map(lambda x: (x[1],(x[0],tokenize(x[2]),x[3])))
ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),(removeQuotes(x[2]),re.sub('\s+','_', x[3]),removeQuotes(x[4]),tokenize(x[5]))))
##TweetsPARSED = Tweets.map(lambda x: (removeQuotes(x[1]),1))
##ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),1))



In [95]:
JoinedRDD = TweetsPARSED.join(ItemsPARSED).map(lambda x: (x[0],(x[1][0]+x[1][1])))
JoinedRDD.take(3)

[('93c42d1806632cb767f9d82e4d1448d4',
  ('LorriePearson',
   ['gucci',
    'gg',
    'marmont',
    'matelass233',
    'leather',
    'chain',
    'wallet',
    'saks',
    'fifth',
    'avenue'],
   '0.4466',
   'Gucci GG Marmont matelassé wallet',
   'Gucci',
   'HK$3600',
   ['bi',
    'fold',
    'coin',
    'wallet',
    'distinguished',
    'signature',
    'double',
    'hardware',
    'calfskin',
    'leather',
    'heat',
    'stamped',
    'achieve',
    'boar',
    'effect',
    'giving',
    'textured',
    'appearance',
    'black',
    'leather',
    'card',
    'slots',
    'compartment',
    'coin',
    'pocket',
    'open',
    'w22cm',
    'h9cm',
    'closed',
    'w11cm',
    'h9cm',
    'brass',
    'hardware',
    'italy'])),
 ('93c42d1806632cb767f9d82e4d1448d4',
  ('LorriePearson',
   ['gucci',
    'gg',
    'marmont',
    'matelass233',
    'leather',
    'chain',
    'wallet',
    'saks',
    'fifth',
    'avenue'],
   '0.4466',
   'Gucci GG Marmont matelassé w

In [96]:
COM_ATTRI_ITEMS = JoinedRDD.map(lambda x: (x[0],list(set(x[1][1]+x[1][6]))))

def item_term(record):
    pairs = [((record[0], value),1) for value in record[1]]
    return (pairs)
##ComAttriRDD = COM_ATTRI_ITEMS.flatMap(lambda x: (item_term(x),1))


##print(ComAttriRDD.take(1))
##COM_ATTRI_ITEMS = COM_ATTRI_ITEMS.reduceByKey(lambda a,b:a+b)
COM_ATTRI_ITEMS.take(3)

[('93c42d1806632cb767f9d82e4d1448d4',
  ['gg',
   'chain',
   'textured',
   'card',
   'double',
   'black',
   'wallet',
   'gucci',
   'heat',
   'effect',
   'appearance',
   'distinguished',
   'bi',
   'closed',
   'brass',
   'coin',
   'avenue',
   'leather',
   'h9cm',
   'pocket',
   'w11cm',
   'fold',
   'w22cm',
   'achieve',
   'fifth',
   'italy',
   'giving',
   'saks',
   'slots',
   'marmont',
   'compartment',
   'calfskin',
   'hardware',
   'matelass233',
   'stamped',
   'signature',
   'open',
   'boar']),
 ('93c42d1806632cb767f9d82e4d1448d4',
  ['interior',
   'gg',
   'chain',
   'card',
   'black',
   'matelasse',
   'wallet',
   'maison',
   'gucci',
   'bevelled',
   'tone',
   'gold',
   '2015',
   'michele',
   'forefront',
   'multiple',
   'fastening',
   'pushed',
   'avenue',
   'leather',
   'zip',
   'fashion',
   'alessandro',
   'helm',
   'fifth',
   'zipped',
   'saks',
   'slots',
   'logo',
   'interlocking',
   'marmont',
   'taken',
   'detai

In [97]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import words
from nltk.corpus import wordnet 

manywords = set(list(words.words()) + list(wordnet.words()))


In [98]:
def english_dic(tokens):
    kept = []
    for t in tokens:
        if (t in manywords):
            kept.append(t)
    return kept

def do_stemming(filtered):
    filtered = english_dic(filtered)
    stemmed = []
    for f in filtered:
        stemmed.append(PorterStemmer().stem(f))
        #stemmed.append(LancasterStemmer().stem(f))
        #stemmed.append(SnowballStemmer('english').stem(f))
    return stemmed

COM_ATTRI_ITEMS = COM_ATTRI_ITEMS.map(lambda x: (x[0],english_dic(x[1])))


In [99]:
Flat_COM = COM_ATTRI_ITEMS.flatMap(lambda x:(item_term(x)))
Flat_COM.reduceByKey(lambda a,b:a+b).take(100)

[(('93c42d1806632cb767f9d82e4d1448d4', 'avenue'), 2),
 (('93c42d1806632cb767f9d82e4d1448d4', 'open'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'bevelled'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'tone'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'gold'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'fastening'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'helm'), 6),
 (('93c42d1806632cb767f9d82e4d1448d4', 'sol'), 4),
 (('93c42d1806632cb767f9d82e4d1448d4', 'rare'), 2),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'new'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'jolly'), 2),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'modal'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'fringe'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'love'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'phrase'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'attitude'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'ready'), 14),
 (('63a52afbb1f6f91ffe90d0fc681104f8', 'fringed'), 14),
 (('63a52afbb1f6f91ffe90d0fc68110

In [100]:
Flat_COM = COM_ATTRI_ITEMS.flatMap(lambda x:(item_term(x)))
Flat_COM.reduceByKey(lambda a,b:a+b).filter(lambda x:x[0][0]=='a4d0adc8f332ec2035dbb55fecba48e2').collect()


[(('a4d0adc8f332ec2035dbb55fecba48e2', 'rectangular'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'medium'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'originally'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'sizes'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'tone'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'gold'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'shoulder'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'black'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'softly'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'plaque'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'strap'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'silver'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'lining'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'detachable'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'hardware'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'adjustable'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'pandora'), 6),
 (('a4d0adc8f332ec2035dbb55fecba48e2', 'pocket'), 6),
 (('a4d0adc8f